In [1]:
import pandas as pd
import numpy as np
import plotly.express as px 
import plotly.graph_objects as go 
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from transformers import BertTokenizer, TFBertModel, BertConfig, TFBertForSequenceClassification

The history saving thread hit an unexpected error (OperationalError('database or disk is full')).History will not be written to the database.


AttributeError: module 'tensorflow.python.util.dispatch' has no attribute 'add_fallback_dispatch_list'

In [3]:
split_df = pd.read_pickle("split_df_vectorised.pkl")

In [151]:
df = pd.read_csv("XYZCorp_LendingData.txt", sep = "\t", low_memory = False)

In [9]:
df["purpose"] = df["purpose"].replace({"home_improvement":"house"})
df["purpose"].value_counts()


debt_consolidation    505392
credit_card           200144
house                  53469
other                  40949
major_purchase         16587
small_business          9785
car                     8593
medical                 8193
moving                  5160
vacation                4542
wedding                 2280
renewable_energy         549
educational              326
Name: purpose, dtype: int64

In [10]:
desc_df = df.dropna(subset = ["desc"])[["desc","purpose"]]

In [35]:
desc_sentences = desc_df["desc"].tolist()
desc_sentences = [re.sub(pattern = "(Borrower added on \d{2}\S\d{2}\S\d{2} >)", string = x,repl = "") for x in desc_sentences]
desc_sentences = [re.sub(pattern = "<br/>", string = x,repl = "<br>") for x in desc_sentences]

In [36]:
split_sentences = [x.lower() for x in desc_sentences]
split_sentences = [x.split("<br>") for x in desc_sentences]
split_sentences = [[y.strip() for y in x] for x in split_sentences]

split_sentences = [[y for y in x if y != ""] for x in split_sentences]
split_sentences = [[y.split(".") for y in x] for x in split_sentences]
split_sentences =  [[item for sublist in x for item in sublist] for x in split_sentences]

split_sentences = [[y.strip() for y in x] for x in split_sentences]
split_sentences = [[y for y in x if y != ""] for x in split_sentences]



In [37]:
split_sentences

[['I need to upgrade my business technologies'],
 ['I plan to use this money to finance the motorcycle i am looking at',
  'I plan to have it paid off as soon as possible/when i sell my old bike',
  'I only need this money because the deal im looking at is to good to pass up',
  'I plan to use this money to finance the motorcycle i am looking at',
  'I plan to have it paid off as soon as possible/when i sell my old bike',
  'I only need this money because the deal im looking at is to good to pass up',
  'I have finished college with an associates degree in business and its takingmeplaces'],
 ['to pay for property tax (borrow from friend, need to pay back) & central A/C need to be replace',
  "I'm very sorry to let my loan expired last time"],
 ['I plan on combining three large interest bills together and freeing up some extra each month to pay toward other bills',
  "I've always been a good payor but have found myself needing to make adjustments to my budget due to a medical scare",
  

In [108]:
for i in range(len(split_sentences)):
    if len(set(split_sentences[i])) != len(split_sentences[i]):
        split_sentences[i] = list(set(split_sentences[i]))
       

In [109]:
split_sentences[1]

['I plan to have it paid off as soon as possible/when i sell my old bike',
 'I plan to use this money to finance the motorcycle i am looking at',
 'I only need this money because the deal im looking at is to good to pass up',
 'I have finished college with an associates degree in business and its takingmeplaces']

In [110]:

split_df = desc_df[["desc","purpose"]].copy()
split_sentences = [[y.strip() for y in x] for x in split_sentences]
split_sentences = [[y for y in x if y != ""] for x in split_sentences]
split_df["desc"] = split_sentences

split_df = split_df.explode("desc")
split_df = split_df.dropna()
split_df = split_df.reset_index(drop = True)

In [111]:
split_df["desc"] = split_df["desc"].apply(lambda x:str(x).lower())
#all lower case, case doesn't give info for this task
split_df["desc"] = split_df["desc"].apply(lambda x:re.sub("[^A-Za-z']+", string = x, repl = " "))
#remove punctuation 
from nltk.corpus import stopwords
en_stop = set(stopwords.words('english'))
split_df["desc_stop"] = split_df["desc"].apply(lambda x:[y for y in x.split(" ") if y not in en_stop])
#removed stopwords


In [84]:
split_df["desc"].apply(lambda x:[y for y in x.split(" ") if y not in en_stop])

0                   [need, upgrade, business, technologies]
1          [plan, use, money, finance, motorcycle, looking]
2             [plan, paid, soon, possible, sell, old, bike]
3              [need, money, deal, im, looking, good, pass]
4          [plan, use, money, finance, motorcycle, looking]
                                ...                        
341178    [senior, year, college, obtaining, bachelors, ...
341179    [qualify, financial, aid, used, stafford, loan...
341180                                    [tuition, approx]
341181                     [, month, courses, left, adds, ]
341182    [need, assistance, cover, tuition, graduation,...
Name: desc, Length: 341183, dtype: object

In [89]:
split_df["desc_stop"].values

array([list(['need', 'upgrade', 'business', 'technologies']),
       list(['plan', 'use', 'money', 'finance', 'motorcycle', 'looking']),
       list(['plan', 'paid', 'soon', 'possible', 'sell', 'old', 'bike']),
       ..., list(['tuition', 'approx']),
       list(['', 'month', 'courses', 'left', 'adds', '']),
       list(['need', 'assistance', 'cover', 'tuition', 'graduation', 'date', 'august', ''])],
      dtype=object)

In [8]:
#obtain word counts
from collections import Counter
cnt = Counter()
words = []
for text in split_df["desc_stop"].values:
    for word in text:
        words.append(word)
        cnt[word] += 1
one_list = [w for w in words if cnt[w] == 1]
#get list of words that only appear once


In [ ]:
df

In [251]:
split_df["desc_stop"].iloc[4]

['finished', 'college', 'associates', 'degree', 'business', 'takingmeplaces']

In [118]:
split_df["desc_rare"]  = split_df["desc_stop"].apply(lambda x:[y for y in x if y not in one_list])
#remove all words that only appear once


In [121]:
split_df.to_pickle("split_df_rare.pkl")

In [2]:
split_df = pd.read_pickle("split_df_rare.pkl")

In [138]:
cores  = multiprocessing.cpu_count()

In [3]:
import gensim.downloader

glove_vectors = gensim.downloader.load('fasttext-wiki-news-subwords-300')

KeyboardInterrupt: 

In [168]:
split_df["desc_stemmed"] = split_df["desc_stemmed"].apply(lambda x:[y for y in x if y != ""])

In [5]:
split_df["desc_rare"] =  split_df["desc_rare"].apply(lambda x:[y for y in x if y != ""])

In [6]:
def embed_words(sentence):
    vector = []
    counter = {}
    for word in sentence:
        counter[word] = cnt[word]
    sentence = sorted(sentence, key=counter.get, reverse = True)
    if len(sentence) > 20:
        sentence = sentence[:20]
    for word in sentence:
        try:
            vector.append(glove_vectors[word])
        except KeyError:
            pass
    return np.array(vector)

In [281]:
sentence = split_df["desc_rare"].iloc[0]
vector = []
counter = {}
for word in sentence:
    counter[word] = cnt[word]
sentence = sorted(sentence, key=counter.get, reverse = True)
sentence

['need', 'business', 'upgrade', 'technologies']

In [282]:
counter

{'need': 12399, 'upgrade': 421, 'business': 6134, 'technologies': 21}

In [9]:
split_df["desc_vectors"] = split_df["desc_rare"].apply(embed_words)

In [11]:
split_df.to_pickle("split_df_vectorised.pkl")

In [4]:
 padding(split_df["desc_vectors"].iloc[0])

array([[ 0.0037601, -0.0085634, -0.010759 , ...,  0.025789 , -0.052145 ,
        -0.042133 ],
       [-0.022481 , -0.045884 ,  0.0011983, ...,  0.0095043, -0.024917 ,
        -0.030083 ],
       [-0.014887 ,  0.066357 , -0.019932 , ..., -0.059062 ,  0.020761 ,
        -0.062035 ],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]])

In [6]:
split_df[split_df['desc_rare'].str.len() == 0]


,desc,purpose,desc_stop,desc_rare,desc_vectors


In [7]:
split_df["desc_vectors_pad"] = split_df["desc_vectors"].apply(padding)

In [4]:
from sklearn.preprocessing import LabelEncoder
split_df = split_df.set_index("purpose").drop(["other","renewable_energy"], axis = 0).reset_index()
lb = LabelEncoder()
split_df["purpose_enc"] = lb.fit_transform(split_df["purpose"].copy()) 

In [296]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse = False)
y_raw = ohe.fit_transform(split_df["purpose"].to_numpy().reshape(-1, 1)) 
y = pd.DataFrame(ohe.fit_transform(split_df["purpose"].to_numpy().reshape(-1, 1)), columns = ohe.categories_)

In [34]:
split_df["desc_vectors_comb"] = split_df["desc_vectors"].apply(lambda x:np.add.reduce(x))
split_df = split_df[split_df['desc_rare'].map(lambda d: len(d)) > 0]
split_df = split_df[split_df['desc_vectors'].map(lambda d: len(d)) > 0]

X = pd.DataFrame(split_df["desc_vectors_comb"].tolist()).to_numpy()

In [37]:
y = split_df["purpose_enc"].astype("category")
y.value_counts()

2     179127
1      63100
4      22653
8      13030
5       9912
0       6518
10      4985
6       3294
7       2656
9       1447
3       1434
Name: purpose_enc, dtype: int64

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 42, stratify = y)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size = 0.2,  random_state = 42, stratify = y_train)

In [26]:
X_train = X_train
y_train = y_train.to_numpy()

In [10]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train
)

In [11]:
import xgboost as xgb

model = xgb.XGBClassifier(objective = 'multi:softprob', num_class = 11)
%timeit model.fit(X_train, y_train, sample_weight = sample_weights)


42min 36s ± 1min 43s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [122]:
import pickle
with open("xgb_nlp_dropped.pkl","wb") as files:
    pickle.dump(model, files)

In [12]:
xgb_predict = model.predict(X_test)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print(accuracy_score(y_test,xgb_predict))


0.4328108774662513


In [48]:
mapping = dict(zip(lb.classes_, range(0, len(lb.classes_))))
mapping

{'car': 0,
 'credit_card': 1,
 'debt_consolidation': 2,
 'educational': 3,
 'house': 4,
 'major_purchase': 5,
 'medical': 6,
 'moving': 7,
 'small_business': 8,
 'vacation': 9,
 'wedding': 10}

In [14]:
from sklearn.metrics import multilabel_confusion_matrix
conf = multilabel_confusion_matrix(y_test, xgb_predict)

In [15]:
conf

array([[[58146,  2184],
        [  802,   500]],

       [[38259, 10889],
        [ 6640,  5844]],

       [[20701,  5008],
        [20976, 14947]],

       [[60497,   835],
        [  251,    49]],

       [[53437,  3663],
        [ 2071,  2461]],

       [[56266,  3353],
        [ 1522,   491]],

       [[59430,  1531],
        [  440,   231]],

       [[60101,   977],
        [  372,   182]],

       [[56185,  2856],
        [ 1108,  1483]],

       [[60698,   661],
        [  193,    80]],

       [[57643,  3000],
        [  582,   407]]], dtype=int64)

In [49]:
from sklearn.metrics import classification_report
report = classification_report(y_test, xgb_predict, labels=[0,1, 2, 3, 4, 5, 6, 7, 8, 9, 10], target_names=['car','credit_card','debt_consolidation','educational','house','major_purchase','medical','moving','small_business','vacation','wedding'])
print(report)

                    precision    recall  f1-score   support

               car       0.02      0.04      0.02      1304
       credit_card       0.21      0.27      0.24     12620
debt_consolidation       0.58      0.32      0.41     35826
       educational       0.01      0.02      0.01       287
             house       0.07      0.10      0.09      4531
    major_purchase       0.03      0.07      0.05      1982
           medical       0.01      0.02      0.01       659
            moving       0.01      0.02      0.01       531
    small_business       0.04      0.07      0.05      2606
          vacation       0.01      0.02      0.01       289
           wedding       0.02      0.06      0.03       997

          accuracy                           0.26     61632
         macro avg       0.09      0.09      0.08     61632
      weighted avg       0.39      0.26      0.30     61632



In [374]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, RMSprop
import tensorflow.keras.backend as K



In [ ]:
model = Sequential()
model.add(Dense(60, input_shape = (20,), activation = "relu"))
model.add(Dense(15, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(13, activation = "softmax"))
model.compile(Adam(lr = 0.01), "categorical_crossentropy", metrics = ["accuracy"])
model.summary()

In [ ]:

model.fit(X_train, y_train)

In [ ]:
set([x.shape for x in X_train ])

In [378]:
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

(None, 20) <dtype: 'float32'>
(None, 13) <dtype: 'float32'>
dense_2 (None, 20) float32
dense_3 (None, 60) float32
dropout (None, 15) float32
dense_4 (None, 15) float32


[None, None, None, None]

In [ ]:
X_train

In [22]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [18]:
import tensorflow_gpu
tensorflow_gpu.__version__

ModuleNotFoundError: No module named 'tensorflow_gpu'

In [24]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

rfc.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [56]:
from sklearn import datasets
iris = datasets.load_iris()

In [58]:
iris.data.shape

(150, 4)

In [59]:
iris.target.shape

(150,)

In [65]:
X_train.to_numpy().reshape((-1,20))

ValueError: cannot reshape array of size 209848 into shape (20)

In [70]:
split_df["desc_vectors_comb"] = split_df["desc_vectors"].apply(lambda x:np.add.reduce(x))

In [33]:
y_test.value_counts()

2     35826
1     12620
4      4531
8      2606
5      1982
0      1304
10      997
6       659
7       531
9       289
3       287
Name: purpose_enc, dtype: int64

In [169]:
df[df["purpose"] == "debt_consolidation"]["desc"].tolist()

['  Borrower added on 12/18/11 > I am planning on using the funds to pay off two retail credit cards with 24.99% interest rates, as well as a major bank credit card with a 18.99% rate.  I pay all my bills on time, looking for a lower combined payment and lower monthly payment.<br>',
 '  Borrower added on 12/15/11 > I had recived a loan from Citi Financial about a year ago, I was paying 29.99 intrest, so the refinance is to cut that rate since cleaning up my credit I have been paying everything on time as shown on my credit report<br>',
 nan,
 '  Borrower added on 12/15/11 > Plan to pay off 2 charge accounts. I will close one of them and ask for a credit line decrease from the other. Also borrowed money from a friend and would like to pay that off.......<br><br>  Borrower added on 12/17/11 > The credit card that I am asking to be decreased will be ONLY for emergency purposes.....<br>',
 nan,
 nan,
 'After amassing credit card debt through several years of college, I now have spending un

In [154]:
with open("y_test.pkl","wb") as files:
    pickle.dump(y_test, files)

In [155]:
with open("y_predict.pkl", "wb") as files:
    pickle.dump(xgb_predict, files)

In [172]:
sample_weights.shape

(197219,)

In [36]:
split_df["purpose"].value_counts()

debt_consolidation    179127
credit_card            63100
house                  22653
small_business         13030
major_purchase          9912
car                     6518
wedding                 4985
medical                 3294
moving                  2656
vacation                1447
educational             1434
Name: purpose, dtype: int64

In [38]:
split_df["purpose_enc"].value_counts()

2     179127
1      63100
4      22653
8      13030
5       9912
0       6518
10      4985
6       3294
7       2656
9       1447
3       1434
Name: purpose_enc, dtype: int64

In [46]:
lb.inverse_transform([3])

array(['educational'], dtype=object)

In [2]:
split_df[["purpose","desc_vectors_comb","desc_rare"]].to_pickle("embeddings2.pkl")

NameError: name 'split_df' is not defined

In [53]:
X_train.shape

(197219, 300)

In [1]:
df

NameError: name 'df' is not defined